# Analyze the fit results from `sncosmo`

Authors: Midori Rollins and Melissa Graham



## Set up and read in the data

Import packages.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os

from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Tcmb0=2.725 * u.K, Om0=0.3)

Read in the `sncosmo` results.

In [ ]:
fnm = '2_all_potential_snia_fits/results_sncosmos.dat'

fit_cand = np.loadtxt(fnm, dtype='str', usecols=(0))
fit_est_z = np.loadtxt(fnm, dtype='float', usecols=(1))
fit_in_zlo = np.loadtxt(fnm, dtype='float', usecols=(2))
fit_in_zhi = np.loadtxt(fnm, dtype='float', usecols=(3))
fit_z = np.loadtxt(fnm, dtype='float', usecols=(4))
fit_ze = np.loadtxt(fnm, dtype='float', usecols=(5))
fit_t0 = np.loadtxt(fnm, dtype='float', usecols=(6))
fit_t0e = np.loadtxt(fnm, dtype='float', usecols=(7))
fit_x0 = np.loadtxt(fnm, dtype='float', usecols=(8))
fit_x0e = np.loadtxt(fnm, dtype='float', usecols=(9))
fit_x1 = np.loadtxt(fnm, dtype='float', usecols=(10))
fit_x1e = np.loadtxt(fnm, dtype='float', usecols=(11))
fit_c = np.loadtxt(fnm, dtype='float', usecols=(12))
fit_ce = np.loadtxt(fnm, dtype='float', usecols=(13))

fit_pmB = np.loadtxt(fnm, dtype='float', usecols=(14))
fit_pmg = np.loadtxt(fnm, dtype='float', usecols=(15))
fit_pmr = np.loadtxt(fnm, dtype='float', usecols=(16))
fit_pmi = np.loadtxt(fnm, dtype='float', usecols=(17))

del fnm

And the review flags.

In [ ]:
fnm = '2_all_potential_snia_fits/MLG_review_ratings.txt'
mlg_review_candid = np.loadtxt(fnm, dtype='str', usecols=(0))
mlg_review_flag = np.loadtxt(fnm, dtype='int', usecols=(1))
del fnm

fit_mlgflag = np.zeros(len(fit_cand), dtype='int') - 1
for c, candid in enumerate(fit_cand):
    tx = np.where(candid == mlg_review_candid)[0]
    if len(tx) == 1:
        fit_mlgflag[c] = mlg_review_flag[tx[0]]
    del tx

In [ ]:
fnm = '2_all_potential_snia_fits/MR_review_ratings.txt'
mr_rating_candid = np.loadtxt(fnm, dtype='str', usecols=(0))
mr_rating_flag   = np.loadtxt(fnm, dtype='int', usecols=(1))
del fnm

fit_mrflag = np.zeros(len(fit_cand), dtype='int') - 1
for c, candid in enumerate(fit_cand):
    tx = np.where(candid == mr_rating_candid)[0]
    if len(tx) == 1:
        fit_mrflag[c] = mr_rating_flag[tx]
    del tx

## A primer on the SALT2 fit parameters

We have used `sncosmo` with the SALT-2 lightcurve fitter,
(<a href="https://ui.adsabs.harvard.edu/abs/2007A%26A...466...11G/abstract">Guy et al. 2007</a>).

The five fit parameters are:
 * `z`  : redshift
 * `t0` : time of peak brightness (MJD)
 * `x0` : flux normalization (a proxy for peak brightness)
 * `x1` : a proxy for lightcurve decline rate (or stretch factor)
 * `c`  : color offset
 
### `z` and `t0`

These two are standard, self-explanatory parameters and do not need any further conversions.

### `x0` 

This is the SED normalization in the SALT2 fits, and a conversion from it to 
peak apparent brightness can be done with 
$m_{B} = −2.5\log(x_{0}) + 10.635$ 
from <a href="https://arxiv.org/abs/1401.4065">Mosher et al. (2014)</a>.

However, this isn't necessary as the peak brightness in Bessel B, and sdss g, r, and i 
are all in the fit results file already.


### `x1`

#### Decline rate

$\Delta m_{15}$ is the change in magnitude (it is standard to report this in the B-band)
during the first 15 days after peak brightness, and is commonly refered to as the
decline rate.

Section 5 of Guy+07 gives the conversion between $x_1$ and $\Delta m_{15}$ as:

> $\Delta m_{15} = 1.09 - 0.161x_{1} + 0.013x_{1}^{2} - 0.00130x_{1}^{3}$

SNIa with a larger value of $\Delta m_{15}$ decline faster, and typically have reached
fainter peak brightness.

#### Stretch

The stretch parameter, $s$, is another parameterization of the width or decline rate
of the SNIa which also incorporates the rise time, and was defined in
<a href="https://ui.adsabs.harvard.edu/abs/1997ApJ...483..565P/abstract">Perlmutter et al. 1997</a>.

Section 5 of Guy+07 gives the conversion between $x_1$ and $s$ as:

> $s = 0.98 + 0.091 x_1 + 0.003 x_1^2 - 0.00075 x_1^3$

SNIa with a larger stretch value have a narrower-width lightcurve that has reached a fainter peak.


### `c` 

Section 2 of Guy+07 describes this as 
*"...a color offset with respect to the average at the date maximum luminosity in B-band, 
$c = (B-V)_{\rm MAX} - \left< B-V \right>$ ..."*.

In this case, $(B-V)_{\rm MAX}$ is the color of the fit SNIa at maximum, 
and $\left< B-V \right>$ is the average color for SNIa in general.

What does the value of $c$ indicate about the fit SNIa's color? Let's reason it out:
 * When $c < 0$, that means $(B-V)_{\rm MAX} < \left< B-V \right>$.
 * The fit SNIa has a $B-V$ value that is lower than average.
 * This means either $B$ is smaller or $V$ is larger.
 * Since we're in magnitudes, that means $B$ is brighter or $V$ is fainter.
 * And $B$ being brighter or $V$ being fainter means the SNIa is *bluer*.
 * So when $c < 0$, the fit SNIa is bluer than a typical SNIa.
 * And vice versa, when $c > 0$ the fit SNIa is redder than a typical SNIa.

This is important to understand because SNIa that are intrinsically brighter are bluer,
and we should see this correlation in our fit results.

## Convert parameters and calculate errors

### Convert `x1` to $\Delta m_{15}$ and stretch

Determine the error values analytically.

#### $\Delta m_{15}$

In [ ]:
fit_dm15 = 1.09 - 0.161*fit_x1 + 0.013*fit_x1**2 - 0.00130*fit_x1**3
fit_dm15e = np.zeros(len(fit_dm15), dtype='float')
for c in range(len(fit_dm15)):
    if fit_x1e[c] > 0:
        temp = np.random.normal(loc=fit_x1[c], scale=fit_x1e[c], size=10000)
        temp2 = 1.09 - 0.161*temp + 0.013*temp**2 - 0.00130*temp**3
        fit_dm15e[c] = np.std(temp2)
        del temp, temp2
    else:
        fit_dm15e[c] = -99.9

#### Stretch

In [ ]:
fit_s = 0.98 + 0.091*fit_x1 + 0.003*fit_x1**2 - 0.00075*fit_x1**3
fit_se = np.zeros(len(fit_s), dtype='float')
for c in range(len(fit_s)):
    if fit_x1e[c] > 0:
        temp = np.random.normal(loc=fit_x1[c], scale=fit_x1e[c], size=10000)
        temp2 = 0.98 + 0.091*temp + 0.003*temp**2 - 0.00075*temp**3
        fit_se[c] = np.std(temp2)
        del temp, temp2
    else:
        fit_se[c] = -99.9

### Calculate error in peak apparent brightness

Derive an error in peak brightness from the error in the fit value of `x0`.

In [ ]:
fit_pmfe = np.zeros(len(fit_x0e), dtype='float')
for c in range(len(fit_x0e)):
    if fit_x0e[c] > 0:
        temp = np.random.normal(loc=fit_x0[c], scale=fit_x0e[c], size=10000)
        temp2 = -2.5 * np.log10(temp) + 10.635
        fit_pmfe[c] = np.std(temp2)
        del temp, temp2
    else:
        fit_pmfe[c] = -99.9

### Estimate peak intrinsic magnitude: $ M_{B}$

Since we have the redshift from the fit, determine the distance modulus and
calculate the peak intrinsic magnitude of each potential SNIa.

The simple way to do this is to use $\mu = m-M$ where $\mu = 5log(D_L) + 25$.

In [ ]:
fit_MB = np.zeros(len(fit_pmB), dtype='float')
tx = np.where(fit_z > 0.0)[0]
fit_MB[tx] = fit_pmB[tx] - (5. * np.log10(cosmo.luminosity_distance(fit_z[tx]).value) + 25.)
del tx

In [ ]:
fit_MBe = np.zeros(len(fit_MB), dtype='float')
for c in range(len(fit_MB)):
    if fit_z[c] > 0.0:
        temp = np.random.normal(loc=fit_z[c], scale=fit_ze[c], size=10000)
        temp2 = np.random.normal(loc=fit_pmB[c], scale=fit_pmfe[c], size=10000)
        temp3 = temp2 - (5. * np.log10(cosmo.luminosity_distance(temp).value) + 25.)
        fit_MBe[c] = np.std(temp3)
        del temp, temp2, temp3
    else:
        fit_MBe[c] = -99.9

Can we use the lightcurve fits to get a "corrected" peak magnitude?

For example, by using this equation from Section 7.2 in Guy+07:

$\mu_B = m^{*}_B - M + \alpha_x \times x_1 - \beta \times c$

This can get you a better *distance* but it relies on the assumption that all SNIa can be "corrected" to a common peak brightness.

Using this equation does not result in a more accurate calculation of the true intrinsic peak brightness.

## Review likely SNIa

Only include potential SNIa with *unflagged* fit parameters in the plots.

In [ ]:
tx = np.where((fit_z > 0.0) & (fit_mlgflag == 0))[0]
tx2 = np.where((fit_z > 0.0) & (fit_mrflag == 0))[0]

txz = np.where(((fit_mlgflag == 0) & (fit_mrflag == 0)) & 
               ((fit_z >= 0.22) & (fit_z <= 0.45)))[0]

print(len(tx), len(tx2), len(txz))

Print the names of just the intermediate redshift likely SNIa.

In [ ]:
print(fit_cand[txz])

Create a grid plot of the intermediate redshift likely SNIa for review.

In [ ]:
clr = ['darkgreen', 'firebrick', 'saddlebrown', 'limegreen', 'orange', 'lightcoral']

all_filters = ['g','r','i']
all_fcolors = ['green','red','blue']
all_fshapes = ['o','s','^']
all_fsizes  = [5, 5, 7]

In [ ]:
fnm = '../all_nightly_epochs_files/candidate_lightcurves.dat'
candLC_candid = np.loadtxt(fnm, dtype='str', usecols=(1))
candLC_mjd = np.loadtxt(fnm, dtype='float', usecols=(3))
candLC_filter = np.loadtxt(fnm, dtype='str', usecols=(4))
candLC_mag = np.loadtxt(fnm, dtype='float', usecols=(6))
candLC_mrb = np.loadtxt(fnm, dtype='float', usecols=(8))
candLC_lef = np.loadtxt(fnm, dtype='float', usecols=(9))
del fnm

In [ ]:
sx = txz
nrow = 5
ncol = 4
fig, ax = plt.subplots( nrow, ncol, figsize=(12, 10), sharex=False, sharey=False )
s = 0
for i in range(nrow):
    ax[i, 0].set_ylabel('magnitude', fontsize=14)
    for j in range(ncol):
        if s <= len(sx)-1:
            for f, filt in enumerate(all_filters):
                cx0 = np.where((candLC_candid == fit_cand[sx[s]]) & \
                               (candLC_filter == filt) & (candLC_mrb > 0.4))[0]
                cx1 = np.where((candLC_candid == fit_cand[sx[s]]) & \
                               (candLC_filter == filt) & (candLC_mrb <= 0.4) & 
                               (candLC_lef == 0))[0]
                cx2 = np.where((candLC_candid == fit_cand[sx[s]]) & \
                               (candLC_filter == filt) & (candLC_mrb <= 0.4) & 
                               (candLC_lef == 1))[0]
                ax[i, j].plot(candLC_mjd[cx0]-59300., candLC_mag[cx0], 
                             all_fshapes[f], ms=all_fsizes[f], mew=0, alpha=0.8, 
                              color=all_fcolors[f], label=filt)
                ax[i, j].plot(candLC_mjd[cx1]-59300., candLC_mag[cx1], 
                             all_fshapes[f], ms=all_fsizes[f], mew=1, alpha=0.4, 
                              color=all_fcolors[f], mec=all_fcolors[f])
                ax[i, j].plot(candLC_mjd[cx2]-59300., candLC_mag[cx2], 
                             all_fshapes[f], ms=all_fsizes[f]-1, mew=1, alpha=0.4, 
                              color='none', mec=all_fcolors[f])
                del cx0, cx1, cx2
            
            cx = np.where((candLC_candid == fit_cand[sx[s]]) & (candLC_lef == 0))[0]
            xmin = np.min(candLC_mjd[cx])
            xmax = np.max(candLC_mjd[cx])
            del cx
            ax[i, j].invert_yaxis()
            ax[i, j].set_xlim([xmin - 59300.0 - 5.0, xmax - 59300.0 + 5.0])
            del xmin, xmax
            ax[i, j].set_title(fit_cand[sx[s]])
            s += 1

ax[0, 0].legend(loc='best', labelspacing=0.1, handletextpad=0.1, fontsize=14)

for j in range(ncol):
    ax[nrow-1, j].set_xlabel('MJD-59300', fontsize=14)

plt.tight_layout()
plt.show()
del s

## Plot parameter distributions

### SALT2 parameters vs redshift

Only `x0`, the peak apparent magnitude, should be correlated with redshift.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 3), sharex=True, sharey=False)

ax[0].errorbar(fit_z[tx], fit_x0[tx], fmt='o', xerr=fit_ze[tx], yerr=fit_x0e[tx], mew=2, color='grey', zorder=1)
ax[0].plot(fit_z[tx2], fit_x0[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[0].set_xlabel('z')
ax[0].set_ylabel('x0')
ax[0].set_ylim([-0.0001,0.00025])
ax[0].axvline(0.22)

ax[1].errorbar(fit_z[tx], fit_x1[tx], fmt='o', xerr=fit_ze[tx], yerr=fit_x1e[tx], mew=2, color='grey', zorder=1)
ax[1].plot(fit_z[tx2], fit_x1[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[1].set_xlabel('z')
ax[1].set_ylabel('x1')
ax[1].axvline(0.22)

ax[2].errorbar(fit_z[tx], fit_c[tx], fmt='o', xerr=fit_ze[tx], yerr=fit_ce[tx], mew=2, color='grey', zorder=1)
ax[2].plot(fit_z[tx2], fit_c[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[2].set_xlabel('z')
ax[2].set_ylabel('c')
ax[2].axvline(0.22)

fig.tight_layout()
fig.show()

Seems to be a degeneracy between x0 and c (peak brightness and color) for lightcurves fit with a low redshift.

Recall _"when $c < 0$, the fit SNIa is bluer than a typical SNIa"_.

So the lightcurves with low redshifts, z<0.22, have been fit as redder (as in, intrinsically redder) than a typical SNIa,
when it's probably more likely they're just at higher redshift (not intrinsically redder but redder due to redshift).

### SALT2 parameters scatter plots

Each parameter vs. the other. We should not see any correlations here, as the three parameters should be independent.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 3), sharex=False, sharey=False)

tx = np.where((fit_z > 0.0) & (fit_mlgflag == 0))[0]
                           
ax[0].errorbar(fit_x0[tx], fit_x1[tx], fmt='o', xerr=fit_x0e[tx], yerr=fit_x1e[tx], mew=2, color='grey', zorder=1)
ax[0].plot(fit_x0[tx2], fit_x1[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[0].set_xlabel('x0')
ax[0].set_xlim([-0.0001,0.00025])
ax[0].set_ylabel('x1')

ax[1].errorbar(fit_x0[tx], fit_c[tx], fmt='o', xerr=fit_x0e[tx], yerr=fit_ce[tx], mew=2, color='grey', zorder=1)
ax[1].plot(fit_x0[tx2], fit_c[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[1].set_xlabel('x0')
ax[1].set_xlim([-0.0001,0.00025])
ax[1].set_ylabel('c')

ax[2].errorbar(fit_x1[tx], fit_c[tx], fmt='o', xerr=fit_x1e[tx], yerr=fit_ce[tx], mew=2, color='grey', zorder=1)
ax[2].plot(fit_x1[tx2], fit_c[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[2].set_xlabel('x1')
ax[2].set_ylabel('c')

fig.tight_layout()
fig.show()

### Derived parameters vs redshift

Here we absolutely expect to see a clear trend between peak apparent magnitude and redshift, 
but not between redshift for stretch or $\Delta m_{15}$.

The approximate expected trend between $m$ and $z$ is shown as a blue line. 
We can see that the "low"-z (z<0.22) and "high"-z (z>0.45) candidates do not lie along this trend.

It is these that are probably contaminating the rest.

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16, 6), sharex=True, sharey=False)

ax[0].errorbar(fit_z[tx], fit_pmB[tx], fmt='o', xerr=fit_ze[tx], yerr=fit_pmfe[tx], mew=2, color='grey', zorder=1)
ax[0].plot(fit_z[tx2], fit_pmB[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[0].set_xlabel('z')
ax[0].set_ylabel('mB')

# add lines
temp_z = 0.10 + np.arange(20, dtype='float') / 40
temp_mB = -19.3 + (5. * np.log10(cosmo.luminosity_distance(temp_z).value) + 25.)
ax[0].plot(temp_z, temp_mB)
ax[0].axvline(0.22)
ax[0].axvline(0.45)
ax[0].set_ylim([20, 23])

ax[1].errorbar(fit_z[tx], fit_s[tx], fmt='o', xerr=fit_ze[tx], yerr=fit_se[tx], mew=2, color='grey', zorder=1)
ax[1].plot(fit_z[tx2], fit_s[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[1].set_xlabel('z')
ax[1].set_ylabel('stretch')

ax[2].errorbar(fit_z[tx], fit_dm15[tx], fmt='o', xerr=fit_ze[tx], yerr=fit_dm15e[tx], mew=2, color='grey', zorder=1)
ax[2].plot(fit_z[tx2], fit_dm15[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[2].set_xlabel('z')
ax[2].set_ylabel('delta m_15')

fig.tight_layout()
fig.show()

### Derived parameters scatter plots

Derived parameters vs. each other (+ color). We should see a trend between delta_m15 and stretch as they were both derived from x1. delta_m15 has a negative relationship with x1, stretch has positive relationship w/ x1, so delta_m15 and stretch should have negative relationship with each other, and that checks out here.

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(15, 10), sharex=False, sharey=False)

ax[0,0].errorbar(fit_pmB[tx], fit_dm15[tx], fmt='o', xerr=fit_pmfe[tx], yerr=fit_dm15e[tx], mew=2, color='grey', zorder=1)
ax[0,0].plot(fit_pmB[tx2], fit_dm15[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[0,0].set_xlabel('mB')
ax[0,0].set_ylabel('delta_m15')

ax[0,1].errorbar(fit_pmB[tx], fit_c[tx], fmt='o', xerr=fit_pmfe[tx], yerr=fit_ce[tx], mew=2, color='grey', zorder=1)
ax[0,1].plot(fit_pmB[tx2], fit_c[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[0,1].set_xlabel('mB')
ax[0,1].set_ylabel('c')

ax[0,2].errorbar(fit_dm15[tx], fit_c[tx], fmt='o', xerr=fit_dm15e[tx], yerr=fit_ce[tx], mew=2, color='grey', zorder=1)
ax[0,2].plot(fit_dm15[tx2], fit_c[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[0,2].set_xlabel('delta_m15')
ax[0,2].set_ylabel('c')

ax[1,0].errorbar(fit_s[tx], fit_c[tx], fmt='o', xerr=fit_se[tx], yerr=fit_ce[tx], mew=2, color='grey', zorder=1)
ax[1,0].plot(fit_s[tx2], fit_c[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[1,0].set_xlabel('stretch')
ax[1,0].set_ylabel('c')

ax[1,1].errorbar(fit_pmB[tx], fit_s[tx], fmt='o', xerr=fit_pmfe[tx], yerr=fit_se[tx], mew=2, color='grey', zorder=1)
ax[1,1].plot(fit_pmB[tx2], fit_s[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[1,1].set_xlabel('mB')
ax[1,1].set_ylabel('stretch')

ax[1,2].errorbar(fit_dm15[tx], fit_s[tx], fmt='o', xerr=fit_dm15e[tx], yerr=fit_se[tx], mew=2, color='grey', zorder=1)
ax[1,2].plot(fit_dm15[tx2], fit_s[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[1,2].set_xlabel('delta_m15')
ax[1,2].set_ylabel('stretch')

fig.tight_layout()
fig.show()

### Intrinsic peak brightness and redshift

Here we **DO NOT** expect to see a trend.

And yet we do, which remains concerning.

What this *might* mean is that a bunch of the lower-redshift objects (z<0.2) are not SNIa.

Or, that their redshifts have been underestimated, and their intrinsic colors (c parameter) overestimated.

It could be a mix.

This trend could be entirely a result of our detection biases, where we can
more easily detect bright things if they're nearby.

In [ ]:
plt.figure(figsize=(6, 6))

plt.errorbar(fit_z[tx], fit_MB[tx], fmt='o', xerr=fit_ze[tx], yerr=fit_MBe[tx], mew=2, color='grey', zorder=1)
plt.plot(fit_z[tx2], fit_MB[tx2], 'o', mew=0, color='orchid', zorder=2)
plt.axvline(0.22)
plt.axvline(0.45)

plt.xlabel('z')
plt.ylabel('intrinsic peak magnitude')
plt.show()

### Lightcurve parameters vs. peak intrinsic magnitude

Here we do expect stretch and $\Delta m_{15}$ to be correlated with peak intrinsic magnitude.

This is commonly known as the Phillips relation, or the width-luminosity relation, from <a href="https://ui.adsabs.harvard.edu/abs/1993ApJ...413L.105P/abstract">Phillips et al. (1993)</a>.

We aren't seeing it super clearly here though....

**Option to redefine indices without the low z candidates:**

In [ ]:
# tx = np.where((fit_z > 0.0) & (fit_flag == 0))[0]
# tx2 = np.where((fit_z > 0.0) & (fit_flagMR == 0))[0]

tx = np.where((fit_z > 0.22) & (fit_mlgflag == 0))[0]
tx2 = np.where((fit_z > 0.22) & (fit_mrflag == 0))[0]

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(15, 7), sharex=True, sharey=False)

ax[0,0].errorbar(fit_MB[tx], fit_c[tx], fmt='o', xerr=fit_MBe[tx], yerr=fit_ce[tx], mew=2, color='grey', zorder=1)
ax[0,0].plot(fit_MB[tx2], fit_c[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[0,0].set_xlabel('MB')
ax[0,0].set_ylabel('c')

ax[0,1].errorbar(fit_MB[tx], fit_s[tx], fmt='o', xerr=fit_MBe[tx], yerr=fit_se[tx], mew=2, color='grey', zorder=1)
ax[0,1].plot(fit_MB[tx2], fit_s[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[0,1].set_xlabel('MB')
ax[0,1].set_ylabel('stretch')

ax[0,2].errorbar(fit_MB[tx], fit_dm15[tx], fmt='o', xerr=fit_MBe[tx], yerr=fit_dm15e[tx], mew=2, color='grey', zorder=1)
ax[0,2].plot(fit_MB[tx2], fit_dm15[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[0,2].set_xlabel('MB')
ax[0,2].set_ylabel('delta m_15')

ax[1,0].errorbar(fit_MB[tx], fit_pmB[tx], fmt='o', xerr=fit_MBe[tx], yerr=fit_pmfe[tx], mew=2, color='grey', zorder=1)
ax[1,0].plot(fit_MB[tx2], fit_pmB[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[1,0].set_xlabel('MB')
ax[1,0].set_ylabel('mB')

ax[1,1].errorbar(fit_MB[tx], fit_x0[tx], fmt='o', xerr=fit_MBe[tx], yerr=fit_x0e[tx], mew=2, color='grey', zorder=1)
ax[1,1].plot(fit_MB[tx2], fit_x0[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[1,1].set_xlabel('MB')
ax[1,1].set_ylabel('x0')

ax[1,2].errorbar(fit_MB[tx], fit_x1[tx], fmt='o', xerr=fit_MBe[tx], yerr=fit_x1e[tx], mew=2, color='grey', zorder=1)
ax[1,2].plot(fit_MB[tx2], fit_x1[tx2], 'o', mew=0, color='orchid', zorder=2)
ax[1,2].set_xlabel('MB')
ax[1,2].set_ylabel('x1')

fig.tight_layout()
fig.show()

As described above, _"when $c < 0$, the fit SNIa is bluer than a typical SNIa"_.

The plot at left of `c` vs. `MB` does show a bluer-brighter trend, which at first is reassuring, but there should be MORE of a trend between `MB` and stretch or `dm15b`. 

## Summary

Still not happy with how this has worked to define a sample of SNIa. Something isn't right in the fits and there's too much correlation between intrinsic peak brightness and `c`.

Although without the low-z candidates the `c` vs `MB` plot looks better (scatter no trend), we don't see the expected trend between stretch and `MB` which is a worry that there is still contamination.